<a href="https://colab.research.google.com/github/AkshayWaghela/FCC/blob/main/AW_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
import seaborn as sns
sns.pairplot(dataset)

In [ ]:
df = dataset

X = df.drop(columns=['expenses'])  # Features
y = np.log(df['expenses'])  # Target

numFeat = ['age', 'bmi', 'children']
catFeat = ['sex', 'smoker', 'region']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numFeat),
        ('cat', OneHotEncoder(), catFeat)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)  # Single output for regression
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mae', metrics=['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_split=0.2,
                    callbacks=[early_stopping, lr_scheduler],
                    verbose=1)

y_pred = np.exp(model.predict(X_test).flatten())  # Reverse log-transform for predictions
y_test_original = np.exp(y_test)  # Reverse log-transform for true values

mae = mean_absolute_error(y_test_original, y_pred)
print(f"Mean Absolute Error: {mae}")


In [ ]:
plt.hist(y_pred)

In [ ]:

test_dataset = X_test  # Already preprocessed during data preparation

# Evaluate the model on the test set (MAE will be computed on the log-transformed scale)
loss, mae_log = model.evaluate(test_dataset, y_test, verbose=2)
print(f"Testing set Mean Abs Error (log scale): {mae_log:.2f}")

# Make predictions and exponentiate to reverse the log-transformation
test_predictions = np.exp(model.predict(test_dataset).flatten())  # Reverse log-transformed predictions
test_labels = np.exp(y_test)  # Reverse log-transformed true values

# Compute MAE on the original scale
mae_original = mean_absolute_error(test_labels, test_predictions)
print(f"Testing set Mean Abs Error (original scale): {mae_original:.2f} expenses")

# Challenge result
if mae_original < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot True Values vs Predictions
plt.figure(figsize=(8, 8))
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions, alpha=0.6)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, max(test_labels.max(), test_predictions.max())]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, color='red')  # Diagonal line indicating perfect predictions
plt.title("True Values vs Predictions (Original Scale)")
plt.show()
